In [1]:
import cv2 
import numpy as np
import os
import tensorflow as tf
from tensorflow import keras
import time 

from skimage.io import imread
from skimage.transform import resize 
from PIL import Image, ImageFont, ImageDraw 
from tensorflow.keras.applications import VGG16
from collections import deque
from tensorflow.keras.models import Model

In [8]:
image_model = VGG16(include_top=True, weights='imagenet')


In [9]:
transfer_layer = image_model.get_layer('fc2')

image_model_transfer = Model(inputs=image_model.input,
                             outputs=transfer_layer.output)
image_model_transfer.summary()

Model: "functional_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)      

In [4]:
model=keras.models.load_model('modelVGG16.h5')
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 512)               9439232   
_________________________________________________________________
dense (Dense)                (None, 1024)              525312    
_________________________________________________________________
activation (Activation)      (None, 1024)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 50)                51250     
_________________________________________________________________
activation_1 (Activation)    (None, 50)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 102       
_________________________________________________________________
activation_2 (Activation)    (None, 2)                 0

In [31]:
input_path=0

output_path='output04.mp4'

In [5]:
input_path="C:/Users/EXTRA/Desktop/F_49_1_2_0_0.mp4"

output_path='output04.mp4'

In [7]:
vid=cv2.VideoCapture(input_path)
fps=vid.get(cv2.CAP_PROP_FPS) # recognize frames per secone(fps) of input_path video file.
print(f'fps : {fps}') # print fps.

writer=None
(W, H)=(None, None)
i=0 # number of seconds in video = The number of times that how many operated while loop .
Q=deque(maxlen=128) 

video_frm_ar=np.zeros((1, int(fps), 244, 224, 3), dtype=np.float) #frames
frame_counter=0 # frame number in 1 second. 1~30
frame_list=[] 
preds=None
maxprob=None

#. While loop : Until the end of input video, it read frame, extract features, predict violence True or False.
# ----- Reshape & Save frame img as (30, 160, 160, 3) Numpy array  -----
while True: 
    frame_counter+=1
    grabbed, frm=vid.read()  # read each frame img. grabbed=True, frm=frm img. ex: (240, 320, 3)
    
    if not grabbed:
        print('There is no frame. Streaming ends.')
        break
            
    if fps!=30: 
        print('Please set fps=30')
        break
        
    if W is None or H is None: # W: width, H: height of frame img
        (H, W)=frm.shape[:2]
            
    output=frm.copy() # It is necessary for streaming captioned output video, and to save that.
    
    frame=resize(frm, (244, 244, 3)) #> Resize frame img array to (160, 160, 3)
    frame_list.append(frame) # Append each frame img Numpy array : element is (160, 160, 3) Numpy array.
    
    if frame_counter>=fps: # fps=30 et al
        #. ----- we'll predict violence True or False every 30 frame -----
        #. ----- Insert (1, 30, 160, 160, 3) Numpy array to LSTM model ---
        #. ----- We'll renew predict result caption on output video every 1 second. -----
        # 30-element-appended list -> Transform to Numpy array -> Predict -> Initialize list (repeat)
        frame_ar=np.array(frame_list, dtype=np.float16) #> (30, 160, 160, 3)
        frame_list=[] # Initialize frame list when frame_counter is same or exceed 30, after transforming to Numpy array.
        print(frame_ar.shape)    
        if(np.max(frame_ar)>1): # Scaling RGB value in Numpy array
            frame_ar=frame_ar/255.0
            
        pred_imgarr=image_model_transfer.predict(frame_ar) #> Extract features from each frame img by using MobileNet. (30, 5, 5, 1024)
        print('pred',pred_imgarr.shape)
        pred_imgarr_dim=pred_imgarr.reshape(1, pred_imgarr.shape[0], 4096)#> (1, 30, 25600)
        print(pred_imgarr_dim.shape)
        
        preds=model.predict(pred_imgarr_dim) #> (True, 0.99) : (Violence True or False, Probability of Violence)
        print(f'preds:{preds}')
        Q.append(preds) #> Deque Q
    
        # Predict Result : Average of Violence probability in last 5 second
        if i<5:
            results=np.array(Q)[:i].mean(axis=0)
        else:
            results=np.array(Q)[(i-5):i].mean(axis=0)
        
        print(f'Results = {results}') #> ex : (0.6, 0.650)
            
        maxprob=np.max(results) #> Select Maximum Probability
        print(f'Maximum Probability : {maxprob}')
        print('')
            
        rest=1-maxprob # Probability of Non-Violence
        diff=maxprob-rest # Difference between Probability of Violence and Non-Violence's
        th=100
            
        if diff>0.80:
            th=diff # ?? What is supporting basis?
        
        frame_counter=0 #> Initialize frame_counter to 0
        i+=1 #> 1 second elapsed
        
        # When frame_counter>=30, Initialize frame_counter to
    # ----- Setting caption option of output video -----
    # Renewed caption is added every 30 frames(if fps=30, it means 1 second.)
    font1=ImageFont.truetype('C:/Users/EXTRA/Desktop/UBI_FIGHTS/Raleway-ExtraBold.ttf', 24) # font option
    font2=ImageFont.truetype('C:/Users/EXTRA/Desktop/UBI_FIGHTS/Raleway-ExtraBold.ttf', 48) # font option
    if preds is not None and maxprob is not None:
        if (preds[0][1])<th : #> if violence probability < th, Violence=False (Normal, Green Caption)
            text1_1='Normal'
            text1_2='{:.2f}%'.format(100-(maxprob*100))
            img_pil=Image.fromarray(output)
            draw=ImageDraw.Draw(img_pil)
            draw.text((int(0.025*W), int(0.025*H)), text1_1, font=font1, fill=(0,255,0,0))
            draw.text((int(0.025*W), int(0.095*H)), text1_2, font=font2, fill=(0,255,0,0))
            output=np.array(img_pil)
                
        else : #> if violence probability > th, Violence=True (Violence Alert!, Red Caption)
            text2_1='Violence Alert!'
            text2_2='{:.2f}%'.format(maxprob*100)
            img_pil=Image.fromarray(output)
            draw=ImageDraw.Draw(img_pil)
            draw.text((int(0.025*W), int(0.025*H)), text2_1, font=font1, fill=(0,0,255,0))
            draw.text((int(0.025*W), int(0.095*H)), text2_2, font=font2, fill=(0,0,255,0))
            output=np.array(img_pil) 
        
    # Save captioned video file by using 'writer'
    if writer is None:
        fourcc=cv2.VideoWriter_fourcc(*'DIVX')
        writer=cv2.VideoWriter(output_path, fourcc, 30, (W, H), True)
            
    cv2.imshow('This is output', output) # View output in new Window.
    writer.write(output) # Save output in output_path
        
    key=cv2.waitKey(round(1000/fps)) # time gap of frame and next frame
    if key==27: # If you press ESC key, While loop will be breaked and output file will be saved.
        print('ESC is pressed. Video recording ends.')
        break
    
print('Video recording ends. Release Memory.')  #Output file will be saved.
writer.release()
vid.release()
cv2.destroyAllWindows()

fps : 30.0
(30, 244, 244, 3)
pred (30, 4096)
(1, 30, 4096)
preds:[[0.97432095 0.02567902]]


<ipython-input-7-f160e6ee2550>:60: RuntimeWarning: Mean of empty slice.
  results=np.array(Q)[:i].mean(axis=0)
C:\Users\EXTRA\anaconda3\envs\ten\lib\site-packages\numpy\core\_methods.py:162: RuntimeWarning: invalid value encountered in true_divide
  ret = um.true_divide(


Results = [[nan nan]]
Maximum Probability : nan

(30, 244, 244, 3)
pred (30, 4096)
(1, 30, 4096)
preds:[[0.97409624 0.02590377]]
Results = [[0.97432095 0.02567902]]
Maximum Probability : 0.9743209481239319

(30, 244, 244, 3)
pred (30, 4096)
(1, 30, 4096)
preds:[[0.97303253 0.0269675 ]]
Results = [[0.9742086 0.0257914]]
Maximum Probability : 0.9742085933685303

(30, 244, 244, 3)
pred (30, 4096)
(1, 30, 4096)
preds:[[0.97417766 0.0258223 ]]
Results = [[0.9738166  0.02618343]]
Maximum Probability : 0.9738165736198425

(30, 244, 244, 3)
pred (30, 4096)
(1, 30, 4096)
preds:[[0.9738221 0.0261779]]
Results = [[0.9739068  0.02609315]]
Maximum Probability : 0.9739068150520325

(30, 244, 244, 3)
pred (30, 4096)
(1, 30, 4096)
preds:[[0.97350806 0.02649198]]
Results = [[0.97388995 0.0261101 ]]
Maximum Probability : 0.973889946937561

(30, 244, 244, 3)
pred (30, 4096)
(1, 30, 4096)
preds:[[0.9743056  0.02569437]]
Results = [[0.97372735 0.02627269]]
Maximum Probability : 0.9737273454666138

(30, 244

In [33]:
vid=cv2.VideoCapture(input_path)
cv2.destroyAllWindows()